In [53]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

'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'

In [54]:
from langchain_groq import ChatGroq
model = ChatGroq(model='gemma2-9b-it', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1093eb0a0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1093eace0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [55]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, i am venkat and i am a software enginner")])

AIMessage(content="Hi Venkat, it's nice to meet you!\n\nThat's great that you are a software engineer.  What kind of software engineering do you specialize in? \n\nI'm always interested in learning more about different areas of tech. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.002093411, 'queue_time': 0.245866038, 'total_time': 0.103911593}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--42cfe0db-00f4-4c1f-a96e-48098cf22f39-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [56]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, i am venkat and i am a software enginner"),
    AIMessage(content="Hi Venkat,\n\nIt's nice to meet you! \n\nSoftware engineering is a fascinating field. What kind of software do you specialize in? \n\nDo you have any projects you're particularly proud of or working on that you'd like to talk about?\n"),
    HumanMessage(content="What is my name?"),
])

AIMessage(content='Your name is Venkat. 😊 \n\nI remembered from our earlier conversation!  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 93, 'total_tokens': 119, 'completion_time': 0.047272727, 'prompt_time': 0.004204333, 'queue_time': 0.243797986, 'total_time': 0.05147706}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d94ecb24-84f3-43d9-8c09-d5dd72fec720-0', usage_metadata={'input_tokens': 93, 'output_tokens': 26, 'total_tokens': 119})

In [57]:
### message history
###we need to make it statefull

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [58]:
config = {"configurable": {"session_id": "chat1"}}

In [59]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, i am venkat and i am a software enginner")], config=config
)

In [60]:
response.content

"Hi Venkat, nice to meet you! 👋\n\nIt's great to know you're a software engineer. What kind of software engineering do you specialize in? 💻  \n\nDo you have any projects you're particularly proud of, or any areas of software development that you're particularly interested in? 😊\n"

In [61]:
with_message_history.invoke([
    HumanMessage(content="What's my name?")
], config=config)

AIMessage(content='Your name is Venkat.  😊 \n\nI remember that from our first exchange!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 103, 'total_tokens': 134, 'completion_time': 0.056363636, 'prompt_time': 0.005419745, 'queue_time': 0.24498034500000002, 'total_time': 0.061783381}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--fe37f44c-e7dd-4fcf-bda0-f9d220dfcaef-0', usage_metadata={'input_tokens': 103, 'output_tokens': 31, 'total_tokens': 134})

In [62]:
#change session
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke([
    HumanMessage(content="Whats my name")
], config=config1)

response.content

'As an AI, I have no memory of past conversations and do not know your name. Please tell me your name! 😊\n'

In [63]:
response = with_message_history.invoke([
    HumanMessage(content=" my name is John")
], config=config1)

response.content

"Hi John, it's nice to meet you! 👋\n\nIs there anything I can help you with today?\n"

In [64]:
response = with_message_history.invoke([
    HumanMessage(content="Whats my name")
], config=config1)

response.content

'Your name is John, as you told me earlier. 😊 \n\nHow can I help you further? \n'

### Prompt templates

In [65]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant, Answer all the question to the nest of your ablility"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [66]:
chain.invoke({"messages": [HumanMessage(content="Hi my name is venkat")]})

AIMessage(content="Hi Venkat! 👋\n\nIt's nice to meet you. 😊\n\nWhat can I do for you today?  I'm ready to answer your questions to the best of my ability. Just ask! \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 33, 'total_tokens': 80, 'completion_time': 0.085454545, 'prompt_time': 0.002292393, 'queue_time': 0.245249516, 'total_time': 0.087746938}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3bf50063-b89b-4bfa-8850-5225da09dd10-0', usage_metadata={'input_tokens': 33, 'output_tokens': 47, 'total_tokens': 80})

In [67]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [68]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke([
    HumanMessage(content="my name is venkat")
], config= config)

response.content

"Hi Venkat!\n\nI'm happy to be your helpful assistant. Ask me anything, and I'll do my best to answer to the best of my ability. 😊  \n\nWhat can I help you with today? \n\n"

In [69]:
response = with_message_history.invoke([
    HumanMessage(content="what is my name ?")
], config= config)

response.content

'Your name is Venkat. 😊 \n\nI remember that from our introduction!  Is there anything else I can help you with?\n'

In [70]:
### adding more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "you are a helpful assistant, Answer all the question to the nest of your ablility in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [71]:
response = chain.invoke({ "messages": [ HumanMessage(content="Hi My name is venkat") ], "language": "Hindi" }, config=config)
response.content

'नमस्ते वेंकट! \n\nमुझे आपकी मदद करने में खुशी होगी। आपके कोई सवाल हैं? \n'

### lets wrap the more complicated chain is a message history class. this time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [72]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [73]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="Hi, I am Venkat")],
        "language": "Hindi"
    },
    config=config
)
response.content

'नमस्ते वेंकट! 👋 \n\nमुझे खुशी है कि आप मेरे साथ बात कर रहे हैं।  आपके सवालों के जवाब देने में मैं अपनी पूरी कोशिश करूंगा। \n\nआप क्या जानना चाहते हैं? 😊\n'

## Managing the conversation History

### add a step that limits the size of the messages you are passing in. 
'trim_messages' reduce how many messages we're sending to the model

In [74]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="Hi, I am bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanila ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanila ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [75]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke({
    "messages": messages + [HumanMessage(content="What icecream do i like")],
    "language": "English"
})

response.content

'You said you like vanilla ice cream! 😊  \n\n\n\nAnything else I can help you with?\n'

In [76]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What math problem did i ask?")],
    "language": "English"
})

response.content

'You asked "What is 2 + 2" 😊 \n\n\n'

In [77]:
### lets wrap this in the message history

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "chat5"}}

In [80]:
response = with_message_history.invoke({
    "messages": messages + [HumanMessage(content="What math problem did i ask?")],
    "language": "English"
}, config=config)

response.content

'You asked "What is 2 + 2". \n\n\n'

In [81]:
response = with_message_history.invoke({
    "messages": messages + [HumanMessage(content="What is my name?")],
    "language": "English"
}, config=config)

response.content

'Your name is Bob.  You told me! 😊  \n\n\n\n'